In [1]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_c0.dta"
)
print(C.shape)

(91319, 21)


In [2]:
# Reduce to the questions of interest (4 questions):

C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scqo4"]
C["currently_enrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

C_relevant = C[
    [
        "hhcode",
        "msno",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrolled",
        "why_not",
        "region",
        "province"
    ]
].copy()


In [3]:
# Fix nas
#C_relevant['currently_enrolled'] = C_relevant['currently_enrolled'].astype('str')
C_relevant['currently_enrolled'] = C_relevant['currently_enrolled'].fillna(value=str('no'))
C_relevant['currently_enrolled'].isna().value_counts()

False    91319
Name: currently_enrolled, dtype: int64

In [4]:
C_relevant

,hhcode,msno,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province
0,1.001100e+09,1,yes,class 10,no,2 education completed,urban,punjab
1,1.001100e+09,2,yes,cl;ass 8,no,2 education completed,urban,punjab
2,1.001100e+09,3,yes,17: ma/msc,no,2 education completed,urban,punjab
3,1.001100e+09,4,yes,class 10,yes,NaN,urban,punjab
4,1.001100e+09,1,yes,12: ba/bsc,no,10 employment,urban,punjab
...,...,...,...,...,...,...,...,...
91314,4.262002e+09,6,no,NaN,no,15 other,rural,balochistan
91315,4.262002e+09,1,no,NaN,no,10 employment,rural,balochistan
91316,4.262002e+09,2,no,NaN,no,13 parent did not approve,rural,balochistan
91317,4.262002e+09,3,no,NaN,no,13 parent did not approve,rural,balochistan


In [5]:
B = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_b0.dta"
)
B.sample(5)

# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['age'] = B['sbq04']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode', 'msno', 'age', 'sex', 'marital_status']].copy()

In [6]:
B.sample(5)

,hhcode,sec,msno,sbq01,sbq02,sbq03,sbq04,sbq05,weight,psu,quarter,region,psu_hies,hhno,hhcode_hies,province,sex,age,marital_status
78421,3.152002e+09,B,6,male,present,3:son/daughter,15,1:never married,309.322998,31520016,3,rural,31520301,3,3.152030e+09,nwfp,male,15,1:never married
59492,2.161103e+09,B,3,male,present,0:non relative,24,1:never married,1705.633057,21611029,4,urban,20511402,12,2.051140e+09,sindh,male,24,1:never married
87265,4.011101e+09,B,6,female,present,6: brother / sister,20,1:never married,242.457993,40111012,4,urban,40711401,9,4.071140e+09,balochistan,female,20,1:never married
51633,2.101100e+09,B,5,female,present,3:son/daughter,8,1:never married,759.708008,21011002,1,urban,20611101,5,2.061110e+09,sindh,female,8,1:never married
64382,3.012001e+09,B,2,female,present,2: husband/wife,24,2:currently married,1623.744995,30120007,4,rural,30120401,14,3.012040e+09,nwfp,female,24,2:currently married


In [7]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [8]:
# MERGE
years04_05 = pd.merge(B_relevant, C_relevant, on=["hhcode", 'msno'], how='inner', indicator=False)

# Evaluate merge 
#years04_05._merge.value_counts()

In [9]:
# Select people in school enrollment age (I would say 4-15 years old)

years04_05 = years04_05.loc[(years04_05["age"] > 4)&(years04_05["age"] <= 15)]
years04_05['year']=2004

In [10]:
# Change dtype to string (hhcode)
years04_05["hhcode"] = years04_05["hhcode"].astype("str")

In [11]:
#years04_05['subprovince']=years04_05['hhcode'].str.extract(r'(^w{3})')
years04_05['subprovince code']= years04_05['hhcode'].apply(lambda x: x[0:4])

In [12]:
years04_05["subprovince code"] = years04_05["subprovince code"].astype("int")

In [13]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0405_v2.csv"
)
strat_name.sample(5)

,province,subprovince code,subprovince
86,Balochistan,4132,Kharan
58,NWFP,3082,Peshawar
40,Sindh,2062,Jaccobabad
100,Punjab,1111,Rawalpindi
97,Balochistan,4242,Jafarabad


In [14]:
# MERGE
years04_05_ = pd.merge(years04_05, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years04_05_._merge.value_counts()

both          25457
left_only         0
right_only        0
Name: _merge, dtype: int64

In [15]:
# add year
years04_05_['year']=2004
years04_05_['marital_status']=years04_05_['marital_status'].astype('str')

#fix marital status categories
years04_05_.loc[(years04_05_['marital_status'] == "1:never married"), 'marital_status'] = 1
years04_05_.loc[(years04_05_['marital_status'] == "2:currently married"), 'marital_status'] = 2
years04_05_.loc[(years04_05_['marital_status'] == "5:nikkah soleminised but rukhsati not taken place"), 'marital_status'] = 5
years04_05_.loc[(years04_05_['marital_status'] == "4:widow / widowers"), 'marital_status'] = 3
years04_05_.loc[(years04_05_['marital_status'] == "3:divoced"), 'marital_status'] = 4
years04_05_['marital_status'].value_counts()
years04_05_['marital_status']=years04_05_['marital_status'].astype('int')

# fix column names
years04_05_['idc']=years04_05_['msno']
years04_05_['province']=years04_05_['province_x']

In [16]:
#select relevant columns
years04_05_ = years04_05_[
    [
        "hhcode",
        "idc",
        "age",
        "marital_status",
        "sex",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        "subprovince code",
        "subprovince",
        "year",
    ]
].copy()

In [17]:
years04_05_.sample(5)

,hhcode,idc,age,marital_status,sex,ever_admitted,currently_enrolled,region,province,subprovince code,subprovince,year
6822,1222002201.0,6,14,1,female,yes,yes,rural,punjab,1222,Vehari,2004
20254,3212000806.0,7,12,1,male,yes,no,rural,nwfp,3212,Bannu,2004
18818,3132001210.0,4,10,1,male,yes,yes,rural,nwfp,3132,Hangu,2004
17560,3082000712.0,6,15,1,female,no,no,rural,nwfp,3082,Peshawar,2004
8048,1272001907.0,10,8,1,male,yes,yes,rural,punjab,1272,Lodhran,2004


In [18]:
years04_05_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years04_05_.csv', index_label="id")

# Part 2

In [19]:
years04_05_["currently_enrolled"].isna().value_counts() # 9608
years04_05_["currently_enrolled"].loc[years04_05_["currently_enrolled"].isna()]

Series([], Name: currently_enrolled, dtype: category
Categories (2, object): ['yes' < 'no'])

In [20]:
# PRE-PROCESSING FOR DIFF-IN-DIFF DATA
# convert currently_enrolled from string to integer
years04_05_["currently_enrolled"].replace("yes", 1, inplace=True)
years04_05_["currently_enrolled"].replace("no", 0, inplace=True)

# check region for anomalies
years04_05_["region"].value_counts()

rural    20738
urban     4719
Name: region, dtype: int64

In [21]:
#years04_05_['currently_enrrolled'] = years04_05_['currently_enrrolled'].astype("int")
years04_05_["currently_enrolled"].isna().value_counts()
years04_05_["currently_enrolled"]= years04_05_["currently_enrolled"].astype('int')

In [22]:
# aggregate dataset for sample population
df_grp_1 = years04_05_.groupby(["sex","subprovince","region"])["currently_enrolled"].count().reset_index()

In [23]:
# aggregate dataset for enrollment total
df_grp_2 = years04_05_.groupby(["sex","subprovince","region"])["currently_enrolled"].sum().reset_index()

In [24]:
# merge data set
df_grp_merge = pd.merge(df_grp_1, df_grp_2, on=["sex","subprovince","region"], indicator=True)
# check merge
df_grp_merge._merge.value_counts()

both          388
left_only       0
right_only      0
Name: _merge, dtype: int64

In [25]:
df_grp_merge.isna().any()

sex                     False
subprovince             False
region                  False
currently_enrolled_x    False
currently_enrolled_y    False
_merge                  False
dtype: bool

In [26]:
# Code Cell 2:
# compute enrollment_rate
df_grp_merge["rate_enrollment"] = df_grp_merge["currently_enrolled_y"]/df_grp_merge["currently_enrolled_x"]

# rename columns
df_grp_merge = df_grp_merge.rename(columns={"currently_enrolled_x":"sample_population", "currently_enrolled_y":"enrolled_total"})

# drop "_merge" column
df_grp_merge.drop(["_merge"], axis=1, inplace=True)
df_grp_merge.sample(5)

,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment
244,female,Hangu,urban,0,0,NaN
349,female,Rajaanpur,rural,93,34,0.365591
317,female,Mir Pur Khas,rural,118,23,0.194915
352,female,Sahiwal,urban,0,0,NaN
23,male,Bonair,rural,108,72,0.666667


In [27]:
df_grp_merge.to_csv("/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/20_analysis/enrollment_clean/years04_05_merge_diff.csv")